# **Preply Web Scrape and Upload to Firebase**

### Task Description
<p>I need to write a python script which will scrape records from a website (each record has about ten text fields and two images which need to be downloaded to a local folder) and save all the text information to a pandas data frame along with the image file names. Each record on the website has a unique id in the url string which can be used as the index/primary key. I think using Selenium with ChromeDriver will be the easiest way to do this because I will have to login through the browser window and navigate to the main page and pick some options before starting the scraping loop.

The second part of the project will be to push the scraped information and images to my firebase project. Since each record has a unique identifier it should be pretty straight forward to use that as the primary key to save the text to firestore. Also the images will need to be uploaded to firebase storage and links to the images saved as a field in firestore for each record.</p>

In [52]:
from selenium.common.exceptions import ElementClickInterceptedException
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from time import sleep
from bs4 import BeautifulSoup
import re
import pandas as pd
import requests # to get image from the web
import shutil # to save it locally
import json

In [54]:
URL = "https://preply.com/en/online/english-tutors?page={0}"

to_int = lambda raw_string: int(''.join(re.findall(r'\d+', raw_string))) # strips anything that is not number and converts to integer

class Selectors:
    tutor_container = "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > li:nth-child({0}) > section"
    currency_button = "#__next > section > div._15uGWh._3rnoor.X4eHUe._1o4EVj.hUFwKI._1-5zEl._2x1aeM._1WH1_Q._2JlASK.GWHMDM._1kf68Q._3nnn2i > div.styles_HeaderDropdownWrapper__uIOg5 > button"
    n_of_pages_selector = "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > div > div > ul > li:nth-child(5) > a"
    tutor = "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > li:nth-child({0})"
    tutor_basic_info = "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > li:nth-child({0}) > section > div > div:nth-child(2) > div.styles_SearchCardHeading__fL9u3"
    tutor_country = "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > li:nth-child({0}) > section > div > div:nth-child(2) > div.styles_SearchCardHeading__fL9u3 > span > img"
    is_newly_joined = "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > li:nth-child({0}) > section > div > div:nth-child(2) > div.styles_CoreIndicatorsBlockWrapper__JrZw6 > div > div > div:nth-child(1) > div > span"
    tutor_rating = "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > li:nth-child({0}) > section > div > div:nth-child(2) > div.styles_CoreIndicatorsBlockWrapper__JrZw6 > div > div > div:nth-child(1) > div"
    thumbnail_img = "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > li:nth-child({0}) > section > div > div.styles_SearchCardAvatarWrapper__u_zNw > div > a > picture > img"
    price = "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > li:nth-child({0}) > section > div > div:nth-child(2) > div.styles_CoreIndicatorsBlockWrapper__JrZw6 > div > div > div:nth-child(2) > div > div.styles_PriceIndicatorPrice__vz9WR > div.styles_PriceIndicatorValue__ndpfb.styles_PriceIndicatorValueSecond__Py5XD"
    lesson_duration = "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > li:nth-child({0}) > section > div > div:nth-child(2) > div.styles_CoreIndicatorsBlockWrapper__JrZw6 > div > div > div:nth-child(2) > div > div.styles_PriceIndicatorDuration__GKnmh"
    lessons_and_students = "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > li:nth-child({0}) > section > div > div:nth-child(3) > div.styles_InfoWrap__CBt_k > div._15uGWh._3rnoor.X4eHUe._1o4EVj.qh7on_._22lwVW._2x1aeM._1WH1_Q._2JlASK.GWHMDM"
    tutor_languages = "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > li:nth-child({0}) > section > div > div:nth-child(3) > div.styles_InfoWrap__CBt_k > div.styles_SpeaksWrapper__ssRz1"
    tutor_languages_more = "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > li:nth-child({0}) > section > div > div:nth-child(3) > div.styles_InfoWrap__CBt_k > div.styles_SpeaksWrapper__ssRz1 > ul > span.styles_MoreSubjects__eJRX9"
    tutor_description = "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > li:nth-child({0}) > section > div > div:nth-child(3) > div.styles_InfoWrap__CBt_k > div.styles_TutorDescriptionBlock__Th9E_ > div > div"
    tutor_description_more = "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > li:nth-child({0}) > section > div > div:nth-child(3) > div.styles_InfoWrap__CBt_k > div.styles_TutorDescriptionBlock__Th9E_ > div > span > span"

def get_total_number_of_pages(driver):
    """Scrapes the total number of pages information
    found at the bottom of the page"""
    total_number_of_pages = int(driver.find_element(By.CSS_SELECTOR, Selectors.n_of_pages_selector).text) + 1
    return total_number_of_pages

def refresh_page_currency(driver):
    """
    Refreshes the page currency from EUR to USD.
    # Context:
    Since I'm in Mexico even that the USD currency is default,
    the page is loaded in MXN pesos currency. To turn around that,
    this function switch to EUR and then to USD, what makes the USD to
    be actually applied.
    """
    driver.find_element(By.CSS_SELECTOR, Selectors.currency_button).click()
    select = Select(driver.find_element(By.NAME, 'currency'))
    sleep(1)
    select.select_by_visible_text("EUR")
    sleep(1)
    driver.find_element(By.CSS_SELECTOR, Selectors.currency_button).click()
    select = Select(driver.find_element(By.NAME, 'currency'))
    select.select_by_visible_text("USD")

def download_thumbnail_img(image_url, file_name):
    ## Set up the image URL and filename
    filename = f"thumbnail_imgs/{file_name}"

    # Open the url image, set stream to True, this will return the stream content.
    response = requests.get(image_url, stream = True)

    # Check if the image was retrieved successfully
    if response.status_code == 200:
        # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
        response.raw.decode_content = True
        
        # Open a local file with wb ( write binary ) permission.
        with open(filename,'wb') as thumbnail_img_file:
            shutil.copyfileobj(response.raw, thumbnail_img_file)
            
        print('Image sucessfully Downloaded: ', filename)
    else:
        print('Image Couldn\'t be retreived')

def write_json(data):
    with open("web_scrape_output.json", "w") as final:
        json.dump(data, final)

def scrape_preply_english_tutors(driver, URL, pages_to_iterate=11):
    """This function iterates over the URL https://preply.com/en/online/english-tutors
    and scrapes all english tutors contained information in it.
    Args:
    *driver: chrome driver object
    *URL: url template that is the scraping target
    *pages_to_iterate: number of pages that will be scraped. Default is 10.
    """
    for page_index in range(1,pages_to_iterate):
        tutors_dicttionary_list = []
        driver.get(URL.format(page_index))
        sleep(3)
        for tutor_index in range(1,20):
            try:
                tutor_basic_info = driver.find_element(By.CSS_SELECTOR,Selectors.tutor_basic_info.format(tutor_index))
                tutor_name = tutor_basic_info.text.split('\n')[0]
                tutor_teaches = tutor_basic_info.text.split('\n')[1]
                tutor_container = driver.find_element(By.CSS_SELECTOR,Selectors.tutor_container.format(tutor_index))
                tutor_id = int(tutor_container.get_attribute("data-qa-id"))
            except NoSuchElementException:
                tutor_basic_info = ""
                tutor_name = ""
                tutor_teaches = ""
                continue
            try:
                driver.find_element(By.CSS_SELECTOR,Selectors.tutor_description_more.format(tutor_index)).click()
                sleep(0.5)
            except:
                print("Description is already fully visible - Nothing to click.")

            try:
                driver.find_element(By.CSS_SELECTOR,Selectors.tutor_languages_more.format(tutor_index)).click()
                sleep(0.5)
            except:
                print("No additional Languages - Nothing to click.")


            try:
                price = driver.find_element(By.CSS_SELECTOR,Selectors.price.format(tutor_index)).text
            except NoSuchElementException:
                price = ""

            try:
                lesson_duration = driver.find_element(By.CSS_SELECTOR,Selectors.lesson_duration.format(tutor_index)).text
            except NoSuchElementException:
                lesson_duration = ""

            try:
                is_newly_joined = driver.find_element(By.CSS_SELECTOR,Selectors.is_newly_joined.format(tutor_index)).text
                is_newly_joined = True
            except NoSuchElementException:
                is_newly_joined = False

            try:
                tutor_rating = driver.find_element(By.CSS_SELECTOR,Selectors.tutor_rating.format(tutor_index)).text
                rating = tutor_rating.split('\n')[0]
                if rating == "Newly joined":
                    rating = ""
                try:
                    n_of_reviews = to_int(tutor_rating.split('\n')[1])
                except:
                    message = "Not rated yet"
                    n_of_reviews = 0
            except NoSuchElementException:
                rating = ""
                message = "Not rated yet"

            try:
                thumbnail_img = driver.find_element(By.CSS_SELECTOR,Selectors.thumbnail_img.format(tutor_index)).get_attribute('src')
            except NoSuchElementException:
                pass

            try:
                tutor_country =  driver.find_element(By.CSS_SELECTOR,Selectors.tutor_country.format(tutor_index)).get_attribute('alt')
            except NoSuchElementException:
                pass

            try:
                lessons_and_students = driver.find_element(By.CSS_SELECTOR,Selectors.lessons_and_students.format(tutor_index)).text
                lessons = to_int(lessons_and_students.split("•")[0])
                try:
                    students = to_int(lessons_and_students.split("•")[1])
                except:
                    students = ""
                    print("No Students yet.")
            except NoSuchElementException:
                lessons = ""

            try:
                languages = driver.find_element(By.CSS_SELECTOR,Selectors.tutor_languages.format(tutor_index)).text
            except NoSuchElementException:
                languages = ""

            try:
                description = driver.find_element(By.CSS_SELECTOR,Selectors.tutor_description.format(tutor_index)).text
            except NoSuchElementException:
                description = ""
                
            thumbnail_img_local_path = f"thumbnail_imgs/{tutor_name.lower().replace(' ','_').replace('.','')}.jpg"
            tutor_dict = \
            { 
             "tutor_id":tutor_id
            ,"tutor_name":tutor_name
            ,"tutor_teaches":tutor_teaches
            ,"thumbnail_img_url":thumbnail_img
            ,"thumbnail_img_local_path":thumbnail_img_local_path
            ,"tutor_country":tutor_country
            ,"price":price
            ,"lesson_duration":lesson_duration
            ,"is_newly_joined":is_newly_joined
            ,"rating":rating
            ,"n_of_reviews":n_of_reviews
            ,"lessons":lessons
            ,"students":students
            ,"languages":languages
            ,"description":description}
            file_name = thumbnail_img_local_path.split('/')[-1]
            download_thumbnail_img(thumbnail_img,file_name)
            print(tutor_dict)
            tutors_dicttionary_list.append(tutor_dict)

    return tutors_dicttionary_list
            
def transform_list_of_dicts_in_df(list_of_dicts):
    "Transforms a list of Dictionaries into pandas DataFrame"
    df = pd.DataFrame(data=list_of_dicts)
    return df

### Scraping Execution

In [55]:
#Initialize Chrome Driver
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("disable-infobars")
chrome_driver_path="C:\Program Files\Google\Chrome\Application\chromedriver.exe"
driver = webdriver.Chrome(options=options)
driver.get(URL.format(1))
sleep(2)
print("Driver successfully initialized.")
test_pages = 2
total_pages = get_total_number_of_pages(driver)
try:
    refresh_page_currency(driver)
    print("Currency successfully refreshed.")
except Exception as e:
    print(f"Could not refresh currency, try again. {e}")
    
list_of_tutor_dictionaries = scrape_preply_english_tutors(driver,URL,test_pages)
write_json(list_of_tutor_dictionaries)
tutors_df = transform_list_of_dicts_in_df(list_of_tutor_dictionaries)



### Pandas DF form

In [48]:
tutors_df.head()

,tutor_id,tutor_name,tutor_teaches,thumbnail_img_url,thumbnail_img_local_path,tutor_country,price,lesson_duration,is_newly_joined,rating,n_of_reviews,lessons,students,languages,description
0,713383,Brianna C.,English,https://avatars.preply.com/i/logos/i/logos/ava...,thumbnail_imgs/brianna_c.jpg,United States of America,30,50-min lesson,True,,0,2,,Speaks:\nEnglishNative Chinese (Mandarin)Begin...,Licensed English teacher with 7 years of exper...
1,3303678,Cristina S.,English,https://avatars.preply.com/i/logos/i/logos/ava...,thumbnail_imgs/cristina_s.jpg,United Kingdom,15,50-min lesson,True,,0,25,,Speaks:\nEnglishNative ItalianPre-Intermediate,Certified TEFL teacher with a creative backgro...
2,3222629,Siriporn K.,English,https://avatars.preply.com/i/logos/i/logos/ava...,thumbnail_imgs/siriporn_k.jpg,Thailand,13,50-min lesson,True,,0,2,,Speaks:\nEnglishProficient ThaiNative,An English teacher with 15 years of experience...
3,123919,Tomasz Ż.,English,https://avatars.preply.com/i/logos/i/logos/888...,thumbnail_imgs/tomasz_ż.jpg,Poland,15,50-min lesson,True,,0,2,,Speaks:\nEnglishProficient PolishNative German...,Certified teacher with 5 years of experience H...
4,3298251,Dave N.,English,https://avatars.preply.com/i/logos/i/logos/ava...,thumbnail_imgs/dave_n.jpg,United Kingdom,15,50-min lesson,True,,0,3,,Speaks:\nEnglishNative RussianUpper-Intermediate,English Speaking Trainer with 10 years experie...
